In [1]:
# The code was removed by Watson Studio for sharing.

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190917172612-0002
KERNEL_ID = a29b1ed9-78b9-40d0-9bfb-eb256557af83


# IEEE CIS Fraud Detection - ETL

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pixiedust.display import *

Pixiedust database opened successfully


In [3]:
# The code was removed by Watson Studio for sharing.

## Training Set

In [51]:
# We have already ETLed the training set in data exploration
df_train = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('transaction_id_train2.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))


In [52]:
df_train.count()

590540

## Test Set

In [6]:
test_identity = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('test_identity.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))

In [7]:
test_transaction = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('test_transaction.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))

In [8]:
test_identity.createOrReplaceTempView("test_identity")
test_transaction.createOrReplaceTempView("test_transaction")

### Joining Transaction and Identity

In [9]:
columns = test_identity.columns[1:]
queryCols = ''
for col in columns:
    queryCols += ', b.'+col
queryCols

', b.id_01, b.id_02, b.id_03, b.id_04, b.id_05, b.id_06, b.id_07, b.id_08, b.id_09, b.id_10, b.id_11, b.id_12, b.id_13, b.id_14, b.id_15, b.id_16, b.id_17, b.id_18, b.id_19, b.id_20, b.id_21, b.id_22, b.id_23, b.id_24, b.id_25, b.id_26, b.id_27, b.id_28, b.id_29, b.id_30, b.id_31, b.id_32, b.id_33, b.id_34, b.id_35, b.id_36, b.id_37, b.id_38, b.DeviceType, b.DeviceInfo'

In [10]:
query = f"""
SELECT 
    a.* {queryCols} 
    FROM test_transaction a
    LEFT JOIN test_identity b
    ON a.transactionID = b.transactionID
"""

transaction_identity = sql(query)
transaction_identity.createOrReplaceTempView("transaction_identity")

In [10]:
test_identity.count()

141907

In [12]:
test_transaction.count()

506691

In [13]:
transaction_identity.count()

506691

In [ ]:
display(transaction_identity)

In [11]:
# select only columns in both training and test
col_train = df_train.columns

In [12]:
col_query = col_train[0]
for col in col_train[1:]:
    if col != 'isFraud':
        col_query = col_query + ', ' + col
col_query

'D13, D14, D12, id_04, id_03, D6, D9, D8, id_09, id_10, id_32, id_14, V153, V161, V162, V163, V138, V157, V156, V155, V154, V139, V149, V158, V147, V146, V142, V141, V140, V148, V152, V151, V150, V145, V144, V143, V159, V160, V165, V164, V166, V322, V332, V339, V338, V337, V336, V335, V334, V333, V331, V323, V329, V328, V327, V326, V325, V324, V330, id_13, V231, V252, V244, V246, V247, V248, V249, V253, V242, V254, V257, V258, V218, V261, V262, V263, V243, V241, V265, V240, V237, V236, V235, V233, V232, V230, V229, V228, V226, V225, V224, V223, V217, V264, V260, V266, V273, V267, V278, V277, V276, V275, V274, V219, V268, V269, id_05, id_06, id_20, id_19, id_17, V177, V213, V179, V178, V176, V214, V173, V172, V168, V215, V216, V167, V212, V202, V181, V203, V211, V207, V183, V186, V187, V206, V191, V196, V205, V204, V193, V199, V192, V182, V190, V188, V174, V184, V189, V180, V194, V195, V197, V198, V200, V201, V175, V185, V170, V208, V209, V210, V169, V171, id_02, id_11, V220, V239, V234

In [13]:
query = f"""
SELECT 
    {col_query}
    FROM transaction_identity
"""

transaction_identity2 = sql(query)
transaction_identity2.createOrReplaceTempView("transaction_identity2")

In [47]:
# save
path = "cos://default-donotdelete-pr-l16zufkrkht8vm.os_938e241641654cf0a8c1bb0aefbea33d_configs/transaction_id_test3.csv"
transaction_identity2.write.format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
    .option('header', 'true').save(path)

## Data Cleansing

In [48]:
df_test = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('transaction_id_test3.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))

In [49]:
# df_train loaded already above
df_train.createOrReplaceTempView("df_train")
df_test.createOrReplaceTempView("df_test")

### Data Types

In [ ]:
display(df_train)

D13,D14,D12,id_04,id_03,D6,D9,D8,id_09,id_10,id_32,id_14,V153,V161,V162,V163,V138,V157,V156,V155,V154,V139,V149,V158,V147,V146,V142,V141,V140,V148,V152,V151,V150,V145,V144,V143,V159,V160,V165,V164,V166,V322,V332,V339,V338,V337,V336,V335,V334,V333,V331,V323,V329,V328,V327,V326,V325,V324,V330,id_13,V231,V252,V244,V246,V247,V248,V249,V253,V242,V254,V257,V258,V218,V261,V262,V263,V243,V241,V265,V240,V237,V236,V235,V233,V232,V230,V229,V228,V226,V225,V224,V223,V217,V264,V260,V266,V273,V267,V278,V277,V276,V275,V274,V219,V268,V269,id_05,id_06,id_20,id_19,id_17,V177,V213,V179,V178,V176,V214,V173,V172,V168,V215,V216,V167,V212,V202,V181,V203,V211,V207,V183,V186,V187,V206,V191,V196,V205,V204,V193,V199,V192,V182,V190,V188,V174,V184,V189,V180,V194,V195,V197,V198,V200,V201,V175,V185,V170,V208,V209,V210,V169,V171,id_02,id_11,V220,V239,V234,V222,V227,V272,V271,V270,V259,V256,V255,V251,V250,V245,V221,V238,id_01,dist1,D5,D2,D11,V11,V6,V2,V10,V9,V8,V7,V5,V1,V4,V3,D3,V43,V50,V49,V48,V47,V46,V45,V44,V39,V42,V40,V35,V36,V52,V37,V38,V41,V51,D4,V85,V93,V92,V91,V90,V89,V88,V87,V86,V84,V83,V81,V80,V79,V78,V77,V76,V75,V94,V82,D15,V74,V72,V70,V69,V68,V67,V66,V65,V64,V63,V62,V61,V60,V59,V58,V57,V56,V55,V54,V53,V73,V71,V18,V22,V34,V31,V32,V17,V28,V27,V26,V25,V24,V23,V33,V21,V12,V16,V30,V15,V14,V29,V13,V19,V20,D10,addr2,addr1,card2,card5,card3,V314,V315,V301,V300,V296,V289,V288,V283,V282,V281,D1,V313,V124,V125,V113,V118,V130,V97,V96,V95,V126,V115,V116,V117,V119,V98,V120,V121,V129,V128,V127,V114,V122,V123,V131,V99,V134,V100,V137,V136,V135,V110,V109,V108,V107,V133,V112,V106,V105,V104,V103,V102,V101,V132,V111,V320,V321,V316,V319,V291,V298,V297,V295,V294,V293,V292,V290,V318,V287,V286,V285,V284,V280,V279,V299,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V317,card1,TransactionAmt,TransactionDT,isFraud,C12,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C14,C13,TransactionID,id_33,id_30,id_34,DeviceInfo,id_16,P_emaildomain,R_emaildomain,id_31,DeviceType,id_29,id_28,id_38,id_35,id_37,id_15,id_36,card4,card6,ProductCD,id_12
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,-480.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,-10.0,222.0,321.0,166.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,68932.0,100.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,87.0,299.0,225.0,224.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2117.0,50.0,773000.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,3018244.0,1920x1080,Android 7.0,match_status:2,SM-G935T Build/NRD90M,NotFound,yahoo.com,gmail.com,chrome 62.0 for android,mobile,NotFound,New,T,T,T,New,F,mastercard,debit,H,NotFound
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [50]:
display(df_test)

D13,D14,D12,id_04,id_03,D6,D9,D8,id_09,id_10,id_32,id_14,V153,V161,V162,V163,V138,V157,V156,V155,V154,V139,V149,V158,V147,V146,V142,V141,V140,V148,V152,V151,V150,V145,V144,V143,V159,V160,V165,V164,V166,V322,V332,V339,V338,V337,V336,V335,V334,V333,V331,V323,V329,V328,V327,V326,V325,V324,V330,id_13,V231,V252,V244,V246,V247,V248,V249,V253,V242,V254,V257,V258,V218,V261,V262,V263,V243,V241,V265,V240,V237,V236,V235,V233,V232,V230,V229,V228,V226,V225,V224,V223,V217,V264,V260,V266,V273,V267,V278,V277,V276,V275,V274,V219,V268,V269,id_05,id_06,id_20,id_19,id_17,V177,V213,V179,V178,V176,V214,V173,V172,V168,V215,V216,V167,V212,V202,V181,V203,V211,V207,V183,V186,V187,V206,V191,V196,V205,V204,V193,V199,V192,V182,V190,V188,V174,V184,V189,V180,V194,V195,V197,V198,V200,V201,V175,V185,V170,V208,V209,V210,V169,V171,id_02,id_11,V220,V239,V234,V222,V227,V272,V271,V270,V259,V256,V255,V251,V250,V245,V221,V238,id_01,dist1,D5,D2,D11,V11,V6,V2,V10,V9,V8,V7,V5,V1,V4,V3,D3,V43,V50,V49,V48,V47,V46,V45,V44,V39,V42,V40,V35,V36,V52,V37,V38,V41,V51,D4,V85,V93,V92,V91,V90,V89,V88,V87,V86,V84,V83,V81,V80,V79,V78,V77,V76,V75,V94,V82,D15,V74,V72,V70,V69,V68,V67,V66,V65,V64,V63,V62,V61,V60,V59,V58,V57,V56,V55,V54,V53,V73,V71,V18,V22,V34,V31,V32,V17,V28,V27,V26,V25,V24,V23,V33,V21,V12,V16,V30,V15,V14,V29,V13,V19,V20,D10,addr2,addr1,card2,card5,card3,V314,V315,V301,V300,V296,V289,V288,V283,V282,V281,D1,V313,V124,V125,V113,V118,V130,V97,V96,V95,V126,V115,V116,V117,V119,V98,V120,V121,V129,V128,V127,V114,V122,V123,V131,V99,V134,V100,V137,V136,V135,V110,V109,V108,V107,V133,V112,V106,V105,V104,V103,V102,V101,V132,V111,V320,V321,V316,V319,V291,V298,V297,V295,V294,V293,V292,V290,V318,V287,V286,V285,V284,V280,V279,V299,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V317,card1,TransactionAmt,TransactionDT,C12,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C14,C13,TransactionID,id_33,id_30,id_34,DeviceInfo,id_16,P_emaildomain,R_emaildomain,id_31,DeviceType,id_29,id_28,id_38,id_35,id_37,id_15,id_36,card4,card6,ProductCD,id_12
0.0,None,692.0,None,None,174.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,27.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0,595.0,266.0,225.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,114129.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-35.0,None,34.0,140.0,None,None,None,None,None,None,None,None,None,None,None,None,34.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,692.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,692.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,None,None,408.0,224.0,185.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,174.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5812,17.214,19580347,2.0,3.0,11.0,0.0,1.0,0.0,1.0,1.0,12.0,0.0,8.0,2.0,3.0,4.0,3698344,None,None,None,None,None,hotmail.com,hotmail.com,mobile safari 11.0,mobile,NotFound,New,F,F,F,Unknown,F,mastercard,debit,C,NotFound
None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,

In [ ]:
# get categorical and numerical columns
cat = ['id_33', 'id_30', 'id_34', 'DeviceInfo', 'id_16', 
       'P_emaildomain', 'R_emaildomain', 'id_31', 'DeviceType', 
       'id_29', 'id_28', 'id_38', 'id_35', 'id_37', 'id_15',
      'id_36', 'card4', 'card6', 'ProductCD', 'id_12'] 
num = [c for c in df_train.columns if c not in cat]

numcat = num + cat
types = ['float' for i in range(len(num))] + ['string' for i in range(len(cat))]
numcat_types = list(zip(numcat, types))
print(numcat_types[:5])
print(numcat_types[-5:])

In [ ]:
# converting numbers from string to float

df_train2 = df_train.select(*(df_train[col].cast(types).alias(col)
                        for col, types in numcat_types))
display(df_train2)

In [ ]:
# converting numbers from string to float

# get categorical and numerical columns
cat = ['id_33', 'id_30', 'id_34', 'DeviceInfo', 'id_16', 
       'P_emaildomain', 'R_emaildomain', 'id_31', 'DeviceType', 
       'id_29', 'id_28', 'id_38', 'id_35', 'id_37', 'id_15',
      'id_36', 'card4', 'card6', 'ProductCD', 'id_12'] 
num = [c for c in df_test.columns if c not in cat]

numcat = num + cat
types = ['float' for i in range(len(num))] + ['string' for i in range(len(cat))]
numcat_types = list(zip(numcat, types))
print(numcat_types[:5])
print(numcat_types[-5:])

df_test2 = df_test.select(*(df_test[col].cast(types).alias(col)
                        for col, types in numcat_types))
display(df_test2)

In [ ]:
df_train2.createOrReplaceTempView('df_train2')
df_test2.createOrReplaceTempView('df_test2')

### Duplicates

In [ ]:
query = """
SELECT COUNT(*), COUNT(DISTINCT(transactionID))
FROM df_train2
"""

sql(query).show()

In [ ]:
query = """
SELECT COUNT(*), COUNT(DISTINCT(transactionID))
FROM df_test2
"""

sql(query).show()

### Null Values

In [ ]:
from pyspark.sql.functions import col, sum

# replace all null values with 0
df_train3 = df_train2.fillna(value=0)

nulls = df_train3.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df_train3.columns))
nulls_pd = nulls.toPandas().transpose()

num_trans = df_train3.count()
nulls_pd.columns = ['counts']
nulls_pd["prop"] = nulls_pd.counts / num_trans
nulls_pd.sort_values(by="prop", ascending=False)

In [ ]:
# replace all null values with 0
df_test3 = df_test2.fillna(value=0)

nulls = df_test3.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df_test3.columns))
nulls_pd = nulls.toPandas().transpose()

num_trans = df_test3.count()
nulls_pd.columns = ['counts']
nulls_pd["prop"] = nulls_pd.counts / num_trans
nulls_pd.sort_values(by="prop", ascending=False)

### Stats Per Numerical Column

In [ ]:
# min, max, mean, stddev

stats_train = df_train3.describe()
display(stats_train)

In [ ]:
# min, max, mean, stddev

stats_test = df_test3.describe()
display(stats_test)

In [ ]:
df_test3.count(), df_train3.count()

In [ ]:
# boxplots on sample?
df_test_sample = df_test3.sample(fraction=5000, seed=0)
df_train_sample = df_train3.sample(fraction=5000, seed=0)

### Categorical Values

In [ ]:
# distinct values

In [ ]:
# case